# Task 1.1: Data understanding

In [ ]:
#Used for displaying plots below the cell
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('customer_supermarket.csv', sep='\t', index_col=0)

In [ ]:
df.head()

The dataset seems to contain data about the shopping habits of the customers of a grocery store chain.  
Each row represents an object purchased:  
- BasketID: identifies a batch of items bought during the same shopping session  
- BasketDate: date in which the shopping session took place  
- Sale: represents the value of the item, we need to figure out if it refers to a single item or the item*quantity
- CustomerID: identifies a unique customer
- CustomerCountry: represents the country in which the purchase took place
- ProdID: identifies a unique product for sale
- ProdDescr: describes the product
- Qta: number of items bought with id ProdID

In [ ]:
df.info()

In [ ]:
len(df.index)

Only ProdDescr and CustomerID contain null values.

In [ ]:
df.describe()

The statistics regarding the CustomerID are meaningless since the assignment of an ID is usually done progressively and without having any additional information on the customer.  
We need to fix the data type situation in order to get a better understanding of the data set.

## Data type conversion  
Let's start by checking out the data type that pandas assigns to the attributes, in order to get an idea of the potential problems.

In [ ]:
df.dtypes

In [ ]:
df.convert_dtypes().dtypes

### CustomerID

CustomerID got converted to a reasonable data type while the others became a generic "string".  
However there is no point in having CustomerID as an int64.

In [ ]:
df["CustomerID"] = df["CustomerID"].astype("object")

### BasketDate
Let's convert the BasketDate type from String to datetime, just in case we need to perform some analysis that requires ordinal data.

In [ ]:
df.BasketDate = pd.to_datetime(df.BasketDate)

### Sale

The "Sale" attribute is considered a generic object while it should be recognised as a float.  
Let's see why.

In [ ]:
df.Sale.map(type)

In [ ]:
df.Sale

It seems that Sale uses a comma instead of a point to separate the decimal part, so it is considered a "str" instead of a "float64".  
Let's replace the commas in "Sale" with dots in order to have them be recognised as float64 by pandas.

In [ ]:
df.Sale = df.Sale.apply(lambda x: x.replace(',','.'))

In [ ]:
df.Sale = df.Sale.astype("float64")

Sale is now correctly identified as a float64.

## Data exploration

### Exploration data frame
Used for exploration purposes but not necessarily useful for clustering.  
Initialised with some additional features that could prove useful.

In [ ]:
#Auxiliary df to be used throughout the data understanding phase
df_expl = df.copy()

df_expl["QtaPositive"] = 0
df_expl.loc[df_expl["Qta"] > 0, "QtaPositive"] = 1 #Indicates whether the records Qta is positive

df_expl["SalePositive"] = 0
df_expl.loc[df_expl["Sale"] > 0, "SalePositive"] = 1 #Indicates whether the records Sale is positive

df_expl.head()

### BasketID

#### Regarding the different types of BasketID
Let's check why BasketID is not considered an int64 like CustomerID.

In [ ]:
nonNumSeries = pd.to_numeric(df.BasketID, errors='coerce').isnull()
# Print the records with BasketIDs containing a non-numeric value
df[nonNumSeries].head()

In [ ]:
df.loc[nonNumSeries, "BasketID"].str.slice(0,1).unique()

It seems that a good chunk of the BasketID values start with a "C" and some with "A" instead of being just numbers.  

In [ ]:
basket_c_df = df.loc[df["BasketID"].str.get(0) == "C"]
len_basket_c = len(basket_c_df)
print(f"Records starting with 'C' (Size: {len_basket_c}):\n")
basket_c_df.head(5)

In [ ]:
basket_a_df = df.loc[df["BasketID"].str.get(0) == "A"]
len_basket_a = len(basket_a_df)
print(f"Records starting with 'A' (Size: {len_basket_a}):\n")
basket_a_df.head(10)

There seems to be a strong correlation between the "C" and a negative quantity, this could indicate a customer that asked for a refund.  

There is also some interesting correlation between the "A" start and a ProdDescr containing "Adjust bad debt", maybe the "A" stands for adjust and since the CustomerID in both cases is NaN this could be an operation that concerns only the management of the shop and not something that concerns the customers (which is our primary objective).  
These records, however, are too few to be meaningful, they skew too much the characteristics of the sale data (outliers) and they don't concern the activities of the customers.

Let's try to add a "BasketID type A" and "BasketID type C" binary attribute (0/1) and see if there are correlations.

In [ ]:
#Initialise all the cells to 0
df_expl["BasketIDTypeA"] = 0
df_expl["BasketIDTypeC"] = 0
df_expl["BasketIDTypeStd"] = 0

#Set the cells appropriately depending on the BasketID type
df_expl.loc[df_expl["BasketID"].str.get(0) == "A", "BasketIDTypeA"] = 1
df_expl.loc[df_expl["BasketID"].str.get(0) == "C", "BasketIDTypeC"] = 1
df_expl.loc[(df_expl["BasketIDTypeA"] == 0) & (df_expl["BasketIDTypeC"] == 0), "BasketIDTypeStd"] = 1

df_expl["NewBasketID"] = df_expl["BasketID"]

#Remove the initial letter from BasketID where necessary
df_expl.loc[df_expl["BasketID"].str.get(0) == "A", "NewBasketID"] = df_expl.loc[(df_expl["BasketID"].str.get(0) == "A"), "BasketID"].str.slice(start=1)
df_expl.loc[df_expl["BasketID"].str.get(0) == "C", "NewBasketID"] = df_expl.loc[(df_expl["BasketID"].str.get(0) == "C"), "BasketID"].str.slice(start=1)

df_expl.corr()

The BasketID of type C has a strong negative correlation with the sign of Qta.

In [ ]:
df_expl.loc[df["BasketID"].str.get(0) == "C", "ProdDescr"].unique()

What could this mean for the C type? Probably indicates discounts/refunds, further checks necessary.

In [ ]:
df_expl["NewBasketID"] = df_expl["NewBasketID"].astype("int64")
df_expl.info()

We notice that there are no more anomalies inside BasketID since it can be now converted to int64.

In [ ]:
df_expl["NewBasketID"] = df_expl["NewBasketID"].astype("string")

Let's check if we now have less unique BasketIDs in our records, after removing the letter that identifies the type from the BasketID attribute.

In [ ]:
print(f'The original number of unique BasketIDs is: {df_expl["BasketID"].unique().size}')
print(f'The current number of unique BasketIDs is: {df_expl["NewBasketID"].unique().size}')

The number is the same, therefore each BasketID of type A or C didn't merge with pre-existing shopping sessions.

#### Regarding the merger of type C records into standard records
Let's check if it is possible/reasonable to associate each type C record with a standard record with the same ProdID, Sale and CustomerID.  

In [ ]:
df_expl["ProdSaleQta"] = df_expl["Qta"]*df_expl["Sale"]
df_expl["AbsProdSaleQta"] = df_expl["ProdSaleQta"].abs()

In [ ]:
#Only about 9000 entries to check, computationally feasible
df_typec = df_expl.loc[df_expl["BasketIDTypeC"] == 1].copy()
std_df = df_expl.loc[df_expl["BasketIDTypeStd"] == 1].copy()

#New dataframe that we are going to save and reuse later for further analysis
type_c_id_list = df_typec["BasketID"].sort_values().unique()
refunds_dict = {}

non_refundable_df = pd.DataFrame(columns=std_df.columns)

i = 0
#Increase sample size for a more accurate analysis
#Set to df_typec.shape[0] for a complete check (Warning: takes a while)
sampleSize = 10

for index, series in df_typec.iterrows():
    tmp_df1 = std_df.loc[(std_df["BasketDate"] <= series["BasketDate"]) & 
                         (std_df["Sale"] == series["Sale"]) & 
                         (std_df["CustomerID"] == series["CustomerID"]) & 
                         (std_df["ProdID"] == series["ProdID"])]
    
    tmp_df1 = tmp_df1.sort_values(["BasketDate"], ascending=False)
    
    #Stronger requirement, if present otherwise merge with a record that satisfies the requirements of tmp_df1
    tmp_df2 = tmp_df1.loc[(tmp_df1["AbsProdSaleQta"] == series["AbsProdSaleQta"])]
    #Whenever in doubt about the API properties...
    tmp_df2 = tmp_df2.sort_values(["BasketDate"], ascending=False)
    
    order_found = None

    if(tmp_df2.empty == False):
        #Given the additional constraint the first entry is sufficient
        order_found = tmp_df2.iloc[0]

        std_df.loc[(std_df["BasketID"] == order_found["BasketID"]) & 
                                (std_df["ProdID"] == order_found["ProdID"]), "AbsProdSaleQta"] = 0
        std_df.loc[(std_df["BasketID"] == order_found["BasketID"]) & 
                                (std_df["ProdID"] == order_found["ProdID"]), "Qta"] = 0
        
        #Save result
        #print(order_found["BasketID"])
        refunds_dict[(series["BasketID"], series["ProdID"])] = pd.array([order_found['BasketID']])
        
    elif(tmp_df1.empty == False):
        order_list = []
        amount_to_refund = -1*series["Qta"]
        tmp_i = 0
        
        while(amount_to_refund > 0 and len(tmp_df1.index) > tmp_i):
            order_found = tmp_df1.iloc[tmp_i]
            tmp_i = tmp_i + 1
            
            if(order_found["Qta"] == 0):
                continue
            
            order_list.append(order_found["BasketID"])
            amount_to_refund = amount_to_refund - order_found["Qta"]
            
            #Need to avoid two different refunds referencing the same order without any items left to be refunded
            if(amount_to_refund >= 0):
                tmp_df1.loc[tmp_df1["BasketID"] == order_found["BasketID"], "Qta"] = 0
                std_df.loc[(std_df["BasketID"] == order_found["BasketID"]) & 
                                (std_df["ProdID"] == order_found["ProdID"]), "Qta"] = 0
            else:
                tmp_df1.loc[tmp_df1["BasketID"] == order_found["BasketID"], "Qta"] = -1*amount_to_refund
                std_df.loc[(std_df["BasketID"] == order_found["BasketID"]) & 
                                (std_df["ProdID"] == order_found["ProdID"]), "Qta"] = -1*amount_to_refund
        
        #Save result
        #print(order_list)
        if(len(order_list) > 0):
            refunds_dict[(series["BasketID"], series["ProdID"])] = pd.array(order_list)
    else:
        #print(f"No match found")
        #print(series)
        #print(std_df.loc[(std_df["CustomerID"] == series["CustomerID"]) & 
        #                 (std_df["Sale"] == series["Sale"]) & 
        #                 (std_df["ProdID"] == series["ProdID"])].head())
        
        non_refundable_df = non_refundable_df.append(series)
        
    #Avoids a long computation on the full dataset
    if(i > sampleSize):
        break
    
    #print(i)
    i = i+1

#Transform a python dictionary with a key of dimension=2 and one value for each key into a DataFrame with 3 columns
#Avoids the problem of inserting lists with different lengths in the dataframe
refund_df = pd.DataFrame(pd.Series(refunds_dict).reset_index()).set_axis(['RefundID', "ProdID",'OrigBasketID'],1,inplace=False)

In [ ]:
print(f"Percentage of entries without a corresponding purchase: {len(non_refundable_df.index)/sampleSize*100}%")

The result calculated on the whole subset type C is : 34.58%

Considering the amount of refund entries for which there is no corresponding purchase there seems to be some missing data or our interpretation of type C records is incorrect.  
For this reason we decide to consider type C records as a generic cost, relative to a specific CustomerID, incurred by the supermarket.  
The other entries with negative Qta will be considered as costs that did not originate from the interaction with a particular customer.  
  
The sum of the entries with negative Qta will henceforth be considered as the TotalCost paid by the company while the sum of everything else is the Revenue.

#### Regarding the shopping habits of a customer that returned for a "refund"

In [ ]:
grouping_cond = ["CustomerID", pd.Grouper(key="BasketDate", freq="D")]

refund_all_cust_df = df_expl.groupby(grouping_cond).filter(lambda x: (x["BasketIDTypeC"] == 1).any())

refund_and_shop_cust_df = df_expl.groupby(grouping_cond)\
            .filter(lambda x: (x["BasketIDTypeC"] == 1).any() and (x["BasketIDTypeC"] == 0).any())

refund_only_cust_df = df_expl.groupby(grouping_cond).filter(lambda x: (x["BasketIDTypeC"] == 1).all())

In [ ]:
refund_all_value = refund_all_cust_df.groupby(grouping_cond).ngroups
refund_and_shop_value = refund_and_shop_cust_df.groupby(grouping_cond).ngroups

print(f"The percentage of times that a customer buys items in the same day when he/she refunds something is: " + \
        f"{refund_and_shop_value/refund_all_value*100}%")

### BasketDate
Let's see how the entries are distributed over time.

In [ ]:
k = math.ceil(math.log(len(df["BasketID"].unique()), 2) + 1) #Sturge's rule


#Since there is only one date for each BasketID take only the first element in each group
df_expl.loc[df_expl["BasketIDTypeStd"] == 1]\
    .groupby(["BasketID"]).nth(0)["BasketDate"].hist(bins=k, figsize=(10,5), alpha=0.5)

df_expl.loc[df_expl["BasketIDTypeC"] == 1].groupby(["BasketID"]).nth(0)["BasketDate"].hist(bins=k, figsize=(10,5), alpha=0.5)

df_expl.loc[(df_expl["BasketIDTypeStd"] == 1) & (df_expl["Qta"] < 0)]\
    .groupby(["BasketID"]).nth(0)["BasketDate"].hist(bins=k, figsize=(10,5), alpha=0.5)


plt.legend(('Standard (All)', 'Type C', 'Standard (Qta < 0)'), loc='best')
plt.title('Distribution of BasketIDs with respect to years')
plt.show()

The number of transactions increases month by month.  
Possibly reflects a change of policy regarding data collection.

In [ ]:
days_dict = dict(zip([0,1,2,3,4,5,6], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]))

basketday = df.groupby(["BasketID"]).nth(0)["BasketDate"].dt.weekday #Series containing the day of the week of each BasketID
basketday = basketday.sort_values().transform(lambda x: days_dict[x])
basketday.hist(bins=np.arange(0, len(days_dict)+1, 1), figsize=(10,5))
plt.title('Distribution of BasketIDs with respect to weekday')
plt.show()

Saturday and sunday seem to be the least active days for the shops.

In [ ]:
months_dict = dict(zip([1,2,3,4,5,6, 7, 8, 9, 10, 11, 12], 
                       ["January", "February", "March", "April", 
                        "May", "June", "July", "August", 
                        "September", "October", "November", "December"]))

basketmonth = df.groupby(by=["BasketID"]).nth(0)["BasketDate"].dt.month #Series containing the day of the week of each BasketID
basketmonth = basketmonth.sort_values().transform(lambda x: months_dict[x]) #Sort needed to have the correct ordering in the plot
basketmonth.hist(bins=np.arange(0, len(months_dict)+1, 1), figsize=(15,5))
plt.title('Distribution of BasketIDs with respect to month')
plt.show()

The sales peak happens in November and the least amount of sales occurs in December.

In [ ]:
#Distributions of Sale and Qta taking into account the BasketDate
fig = plt.figure(figsize=(20, 5)) 
fig_dims = (1, 2)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))

plt.scatter(df_expl["BasketDate"], df_expl["Sale"], color='g', marker='*', label='Standard')

plt.xlabel('BasketDate')
plt.xticks(rotation='vertical')
plt.ylabel('Sale')

plt.subplot2grid(fig_dims, (0, 1))

plt.scatter(df_expl["BasketDate"], df_expl["Qta"], color='g', marker='*', label='Standard')

plt.xlabel('BasketDate')
plt.xticks(rotation='vertical')
plt.ylabel('Qta')

plt.show()

Here are clearly visible data points that could be classified as outliers.

### Sale

In [ ]:
df_expl["Sale"].describe()

#### Regarding negative Sale entries

In [ ]:
df_expl[df_expl["Sale"] < 0].head()

The only entries with a negative Sale are the type A entries, of which there are only two in the entire dataset.

#### Regarding the relationship between Sale and Qta
We need to figure out if the Sale value refers to the cost of a single item or cost of item * Qta

In [ ]:
df.sort_values(by="ProdID").head()

Just by looking at this view of the Data Frame it seems that Sale doesn't change if the Qta changes.

In [ ]:
df.corr()

There is no significant correlation between Sale and Qta, we can therefore assume that Sale is the cost of the single item.

In [ ]:
#Visualize the Sale distribution
fig = plt.figure(figsize=(15, 5)) 
fig_dims = (1, 2)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))
k = math.ceil(math.log(len(df["Sale"].unique()), 2) + 1) #Sturge's rule
df["Sale"].hist(bins=np.arange(0, k, 1))

plt.subplot2grid(fig_dims, (0, 1))
df.boxplot(column=["Sale"])
plt.show()

As expected the vast majority of Sale values (the price for a single item in a transaction) are small.  
We need however to check for 0 values since they don't make sense in the contest of Sale and therefore should be considered as missing values.

In [ ]:
n_missing_sale = df.loc[df["Sale"] == 0, "Sale"].size
n_tot_sale = df["Sale"].size
print(f"Percentage of entries with Sale equal to 0: {(n_missing_sale/n_tot_sale)*100}%")

A small percentage, relative to the entire dataset, of the entries presents a Sale value equal to 0.  
We will take care of them in the Data Preparation phase.

### CustomerID

#### Regarding null CustomerIDs
Let's see why the number of non-null CustomerID entries is so low and if there are any interesting properties to be found.

In [ ]:
df_expl.loc[df_expl["CustomerID"].isna()].describe()

In [ ]:
df_expl["CustomerIDNull"] = 0
df_expl.loc[df_expl["CustomerID"].isna(), "CustomerIDNull"] = 1

In [ ]:
df_expl.corr()["CustomerIDNull"]

No interesting correlation.  
Let's check if we can retrieve some missing CustomerIDs by using the records referencing the same BasketID.

In [ ]:
df_expl.groupby(by="BasketID").filter(lambda x: x["CustomerID"].isna().any() & x["CustomerID"].notna().any()).size

There seems to be no intersection between records with the same BasketID but different CustomerIDNull value.  
We can start to assume that the entries with CustomerID null are done on purpouse and have a specific meaning.  
Further analysis in the Qta section.

#### Regarding entries with missing values in Sale and CustomerID  null
Let's see if all the entries with Sale equal to 0 have also a missing CustomerID.

In [ ]:
zero_sale_null_cust = df_expl.loc[(df_expl["CustomerIDNull"] != 1) & (df_expl["Sale"] == 0)]
zero_sale_null_cust.shape[0]

There is a small amount of entries with both a Sale equal to 0 and a non-Null CustomerID.  
(TODO?) (Check the percentage of Sale values present in other entries?)

In [ ]:
zero_sale_null_cust.describe()

Note that, as expected, the Qta is always positive for these entries.

### Customer country

In [ ]:
#Only one BasketID considered independently from the number of single entries
basketid_country = df_expl.groupby(by=["CustomerCountry"])["BasketID"].nunique()

fig = plt.figure(figsize=(20, 5)) 
fig_dims = (1, 2)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))

basketid_country.plot(kind='bar')
plt.title("Number of unique BasketIDs by country with UK")

plt.subplot2grid(fig_dims, (0, 1))

basketid_country.loc[basketid_country.index != "United Kingdom"].plot(kind='bar')
plt.title("Number of unique BasketIDs by country without UK")

plt.show()

The majority of the singular operations take place in the United Kingdom.  
Any further relevant analysis would require to solve the missing values problem first in order to have meaningful results.

### ProdID

#### ProdID interpretation  
Let's find out why this wasn't converted to a number.

In [ ]:
df.loc[df["ProdID"].str.isnumeric(), ("ProdID", "ProdDescr")].value_counts() #Records with ProdIDs containing only numbers

In [ ]:
df.loc[df["ProdID"].str.isalpha(), ("ProdID", "ProdDescr")].value_counts() #Records with ProdIDs containing only letters

In [ ]:
#Records with ProdID terminating with a letter
term_letter_prodid = df.loc[(df["ProdID"].str.slice(start=-1).str.isalpha()) & (df["ProdID"].str.slice(0, -1).str.isnumeric())]
term_letter_prodid[["ProdID", "ProdDescr"]].sort_values(by="ProdID").value_counts()

The letters seem to represent different variations of the same item.  

Given the diversity and lack of structure of the ProdIDs, as can be seen in the different types listed above, there doesn't seem to be anything worth exploring.

#### Regarding the possibility of having multiple entries for the same (BasketID, ProdID) entry

In [ ]:
ambiguous_df = df_expl.groupby(["BasketID", "ProdID"]).filter(lambda x: x.shape[0] > 1)
ambiguous_df.sort_values(["BasketID", "ProdID"]).head()

In [ ]:
print(ambiguous_df.shape[0])

It shouldn't matter too much for the purpouse of our analysis but it could prove tricky in other situations.  
Note that for this reason the tuple (BasketID, ProdID) is not a "Key" of the dataset.

#### Regarding ProdIDs in type C BasketID records

In [ ]:
df_expl.loc[(df_expl["BasketIDTypeC"] == 1)].groupby(["ProdID"]).apply(lambda x: x["Qta"].sum()).sort_values()

These are the most returned products by quantity.

In [ ]:
df_expl.loc[(df_expl["BasketIDTypeC"] == 1)].groupby(["ProdID"]).apply(lambda x: x["ProdSaleQta"].sum()).sort_values()

These are the most returned products by total sale value.  
The presence of items such as AMAZONFEE and M (manual adjustments) seem to further justify our choice of not considering each type C entry as connected with a standard type one.

### Qta

In [ ]:
df["Qta"].describe()

#### Regarding missing values

In [ ]:
k = math.ceil(math.log(len(df["Qta"].unique()), 2) + 1) #Sturge's rule
df["Qta"].hist(bins=np.arange(0, k))
plt.show()

Let's check for 0 values.

In [ ]:
df.loc[df["Qta"] == 0].size

There are no records with Qta equal to 0, so we can assume that there are no records with missing values in the feature Qta.

#### Regarding negative Qta and type C BasketID

In [ ]:
df_expl.corr()["QtaPositive"]

As noted in the BasketID section there is a strong correlation between the sign of Qta and a BasketID of type C.  
Let's see if there is some interesting distribution in the remaining negative quantities.

In [ ]:
neg_not_c = df_expl.loc[(df_expl["Qta"] < 0) & (df_expl["BasketIDTypeC"] == 0)]
neg_not_c.head()

Let's check if the trend of Sale equal 0 continues throughout the subset of records.

In [ ]:
neg_not_c["Sale"].describe()

It does.  
Let's check if all CustomerIDs in the subset are Null.

In [ ]:
neg_not_c.describe()["CustomerIDNull"]

They are all Null, as can be deduced by the min value.  
All entries not of type C and with a negative Qta have a Sale value equal to 0 and a CustomerID null.  

#### Regarding the intersection between ProdID entries with positive and negative Qtas

In [ ]:
set_neg = set(df.loc[df["Qta"] < 0, "ProdID"])
set_pos = set(df.loc[df["Qta"] > 0, "ProdID"])
inter_list = list(set_pos & set_neg)
print(len(inter_list)/len(set_pos))
print(len(inter_list)/len(set_neg))

In [ ]:
set_neg = set(df.loc[(df["Qta"] < 0) & (df["CustomerID"].notna()), "ProdID"])
set_pos = set(df.loc[(df["Qta"] > 0) & (df["CustomerID"].notna()), "ProdID"])
inter_list = list(set_pos & set_neg)
print(len(inter_list)/len(set_pos))
print(len(inter_list)/len(set_neg))

In both cases there are some values of ProdID present in a particular subset of the dataset that are not present in the other.  
Result needed to justify decision in customer features.

# Task 1.2: Data preparation

In [ ]:
orig_df = df.copy() #Just in case

## Data cleanup   
(Pointless for now since the customer features consider only CustomerID non-null entries)

Remove type A BasketID entries since, as noted above, they don't specify a CustomerID and are concerned with the grocery store chain debt.

In [ ]:
#df = df.drop(df.loc[df["BasketID"].str.get(0) == "A"].index)

Remove records with negative Qta that do not reference a type C BasketID since they do not reference any CustomerID.  
Now all records with negative Qta are of type C.

In [ ]:
#df = df.drop(df.loc[(df["BasketID"].str.get(0) != "C") & (df["Qta"] < 0)].index)

Since we don't seem to have a way to retrieve the CustomerID in case it is missing let's delete all entries without CustomerID.

In [ ]:
#df = df.drop(df.loc[df["CustomerID"].isna()].index)

## Missing values

In [ ]:
df.head()

In [ ]:
missing_df = df.loc[df["Sale"] == 0]
missing_df.shape[0]

Let's try to retrieve the most accurate value possible for the missing Sale.  
We will look for values present in the dataset that satisfy the maximum number of constraints. (TODO: test for better alternatives with models)

In [ ]:
i=0

missed_df = pd.DataFrame(columns=missing_df.columns)

for index, series in missing_df.iterrows():
    
    base_df = df.loc[df["ProdID"] == series["ProdID"]]
    
    #print(i)
    i = i+1
    
    if(base_df.size == 0):
        print("No match found")
        missed_df = missed_df.append(series)
        continue
    
    base_res = base_df["Sale"].median()
    
    lv1_df = base_df.loc[base_df["CustomerCountry"] == series["CustomerCountry"]]
    lv1_res = lv1_df["Sale"].median()
    
    if(lv1_df.size == 0):
        df.loc[(df["BasketID"] == series["BasketID"]) & (df["ProdID"] == series["ProdID"]), "Sale"] = base_res
        continue
        
    df.loc[(df["BasketID"] == series["BasketID"]) & (df["ProdID"] == series["ProdID"]), "Sale"] = lv1_res
    
    #TODO: add a BasketDate constraint?
    #TODO: use K-nn?
    
print(f"The number of entries for which a match wasn't found is: {missed_df.shape[0]}")

The missing values will be replaced by the median of the Sale values obtained by grouping with respect to the ProdID first and then, if possible, with respect to CustomerCountry.  
We used the median instead of the mean in order to be less susceptible to outliers.

## Outlier analysis
Outliers in this dataset are expected to be found in the numerical attributes, therefore we will focus on Sale and Qta.

In [ ]:
plt.scatter(df["Sale"], df["Qta"], color='g', marker='*', label='Standard')
plt.xlabel("Sale")
plt.ylabel("Qta")
plt.show()

### Statistical approach
As seen in the respective sections the data doesn't seem to have a normal distribution.

### IQR approach

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
print(f"Qta 2.5-quantile: {df['Qta'].quantile(0.025)}")
print(f"Qta 97.5-quantile: {df['Qta'].quantile(0.975)}")
print(f"Sale 2.5-quantile: {df['Sale'].quantile(0.025)}")
print(f"Sale 97.5-quantile: {df['Sale'].quantile(0.975)}")

It seems, from the scatter plot and the quantiles, that by using the IQR approach we could lose interesting information that doesn't diverge all that much from the rest, both visually and numerically, for example we would lose any Sale value greater then 12.75 while we can see that there are many data points with a greater value that are not that different (ie 99-quantile of Sale = 16.95).  

Note also that by removing the outliers in Qta with the IQR approach we will also remove all entries having a negative Qta.  
We need to consider if that data is worth keeping.  
The same concerns do not apply to Sale, as seen in the Sale section above.

### Clustering approach

Unsuitable given the size of the dataset (considering DBSCAN).

## Outlier removal
Using the IQR method.

In [ ]:
df_wo_out = df.copy()

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

Remove the outliers in Sale and Qta.

In [ ]:
df_wo_out = df_wo_out.loc[(df_wo_out["Sale"] >= Q1["Sale"] - 1.5*IQR["Sale"]) & (df_wo_out["Sale"] <= Q3["Sale"] + 1.5*IQR["Sale"])]
df_wo_out = df_wo_out.loc[(df_wo_out["Qta"] >= Q1["Qta"] - 1.5*IQR["Qta"]) & (df_wo_out["Qta"] <= Q3["Qta"] + 1.5*IQR["Qta"])]

In [ ]:
df_w_out = df

## Customer features

Let's define some new features to be used for customer analysis.

In [ ]:
def add_features(dest_df, source_df):

#Required features

    #Modify the data frame locally
    source_df = source_df.copy()
    
    #Additional useful data for features
    source_df["DateYear"] = source_df["BasketDate"].dt.year
    source_df["DateMonth"] = source_df["BasketDate"].dt.month
    source_df["DateDay"] = source_df["BasketDate"].dt.day
    source_df["DateWeekDay"] = source_df["BasketDate"].dt.weekday
    
    #Split the dataset into positive Qta and negative Qta in order to simplify notation
    #Note 1: CustomerID null are filtered out by groupby.
    pos_df = source_df.loc[source_df["Qta"] > 0]
    neg_df = source_df.loc[source_df["Qta"] < 0]
    
    #Total number of items bought by customer
    # Since we cannot create a clear association between the refunds and purchases,
    #  we decided to not include the negative Qta entries (type C) in this feature.
    # The negative Qtas will be considered in a separate feature. 
    IFeature = pos_df.groupby(["CustomerID"]).Qta.sum()
    dest_df = dest_df.merge(IFeature, on="CustomerID").rename(columns={"Qta":"I"})

    #Total number of unique items bought by customer
    #As seen in the Qta section, we need to limit the counting to the positive Qta entries,
    # otherwise we also will count some entries for which there is no recorded purchase entry.
    IuFeature = pos_df.groupby(["CustomerID"]).ProdID.nunique()
    dest_df = dest_df.join(IuFeature, on="CustomerID").rename(columns={"ProdID":"Iu"})

    #Max number of item bought by customer across all shopping sessions
    BasketIDQtaSum = pos_df.groupby(["CustomerID", "BasketID"]).Qta.sum()
    ImaxFeature = BasketIDQtaSum.groupby(["CustomerID"]).max()
    dest_df = dest_df.join(ImaxFeature, on="CustomerID").rename(columns={"Qta":"Imax"})
    
    #The Shannon entropy on the purchasing behaviour of the customer: types of items bought
    #Since we are assuming that the type C BasketID entries are a generic cost associated to a CustomerID,
    # we can ignore them for the purpouse of computing the entropy since they might refer to a ProdID that cannot
    # be bought Ie AMAZONFEE
    
    probSeriesGrouped = pos_df.groupby(["CustomerID"])\
            .apply(lambda x: x.groupby(["ProdID"])["Qta"].sum()/x["Qta"].sum())
    logSeriesGrouped = np.log2(probSeriesGrouped)
    ProdProbLogSeriesGrouped = -1 * probSeriesGrouped * logSeriesGrouped
    EFeature = ProdProbLogSeriesGrouped.groupby(["CustomerID"]).sum()
    EFeature = EFeature.fillna(0)
    EFeature.name = "Eproduct"
    dest_df = dest_df.join(EFeature, on="CustomerID")

#Additional features

    #Average total revenue per month
    #Revenue doesn't take into account the costs (negative Qta)
    tot_revenue_cust = pos_df.groupby(["CustomerID"]).apply(lambda x: (x["Sale"]*x["Qta"]).sum())
    n_month_cust = pos_df.groupby(["CustomerID"]).apply(lambda x: x.groupby(["DateYear", "DateMonth"]).ngroups)
    AvgFeature = tot_revenue_cust/n_month_cust
    AvgFeature.name = "AvgRevenueMonth"
    dest_df = dest_df.join(AvgFeature, on="CustomerID")
    
    #Average total costs per month
    #Note: since the entries with costs are a small amount compared to all the others,
    # some customers will have NaN as value. We will replace it with 0.
    tot_costs_cust = neg_df.groupby(["CustomerID"]).apply(lambda x: (x["Sale"]*x["Qta"]).sum())
    n_month_cust = neg_df.groupby(["CustomerID"]).apply(lambda x: x.groupby(["DateYear", "DateMonth"]).ngroups)
    AvgFeature = (tot_costs_cust/n_month_cust).abs()
    AvgFeature.name = "AvgCostsMonth"
    dest_df = dest_df.join(AvgFeature, on="CustomerID")
    dest_df["AvgCostsMonth"] = dest_df["AvgCostsMonth"].fillna(0)

    #Average customer shopping sessions per month
    #We are interested in finding out about the habits of a customer and given the percentage
    # of type C entries compared to standard ones it seems to us that costs are an unusual event
    # rather than the norm. Therefore we exclude the type C entries from our definition of shopping session.
    tot_sessions_cust = pos_df.groupby(["CustomerID"]).apply(lambda x: x["BasketID"].nunique())
    n_month_cust = pos_df.groupby(["CustomerID"]).apply(lambda x: x.groupby(["DateYear", "DateMonth"]).ngroups)
    AvgFeature = tot_sessions_cust/n_month_cust
    AvgFeature.name = "AvgSessionsMonth"
    dest_df = dest_df.join(AvgFeature, on="CustomerID")

    #The Shannon entropy on the purchasing behaviour of the customer: days of the week of shopping
    #Justification for type C entries same as for other entropy related feature.
    probSeriesGrouped = pos_df.groupby(["CustomerID"])\
        .apply(lambda x: x.groupby(["DateWeekDay"])["BasketID"].nunique()/x["BasketID"].nunique())
    logSeriesGrouped = np.log2(probSeriesGrouped)
    ProdProbLogSeriesGrouped = -1 * probSeriesGrouped * logSeriesGrouped
    EFeature = ProdProbLogSeriesGrouped.groupby(["CustomerID"]).sum()
    EFeature.name = "Eweekday"
    dest_df = dest_df.join(EFeature, on="CustomerID")
    
    return dest_df

Apply the new features to both the dataset with and without outliers.

In [ ]:
unq_cust_id = df_w_out["CustomerID"].sort_values().unique()[0:-1] #Remove NaN value, last value
cust_df_w_out = pd.DataFrame(data=unq_cust_id, columns=["CustomerID"]) #Dataframe containing customer features with outliers

unq_cust_id = df_wo_out["CustomerID"].sort_values().unique()[0:-1] #Remove NaN value, last value
cust_df_wo_out = pd.DataFrame(data=unq_cust_id, columns=["CustomerID"]) #Dataframe containing customer features without outliers

cust_df_w_out["CustomerID"] = cust_df_w_out["CustomerID"].astype("object")
cust_df_wo_out["CustomerID"] = cust_df_wo_out["CustomerID"].astype("object")

cust_df_w_out = add_features(cust_df_w_out, df_w_out)
cust_df_wo_out = add_features(cust_df_wo_out, df_wo_out)

## Feature analysis

In [ ]:
cust_df_w_out.describe()

In [ ]:
cust_df_wo_out.describe()

### Correlations

In [ ]:
#Display correlations that have an absolute value greater than threshold
corr_w_out = cust_df_w_out.corr()

threshold = 0.5

for col in corr_w_out.columns:
    corr_w_out[col] = corr_w_out[col].transform(lambda x: x if abs(x)>threshold else 0)

corr_w_out

Considering the outliers, we have a strong correlation between the maximum number of items bought in a single order by a customer and the average revenue per month.  
Other potentially interesting correlations are the ones between the average revenue and average costs, and between Imax and average costs.

In [ ]:
#Display correlations that have an absolute value greater than threshold
corr_wo_out = cust_df_wo_out.corr()

threshold = 0.5

for col in corr_wo_out.columns:
    corr_wo_out[col] = corr_wo_out[col].transform(lambda x: x if abs(x)>threshold else 0)

corr_wo_out

It seems that there is a significant correlation between the max number of items bought and the average expenditure per month of the customer.  
This could suggest to us that the customers with the biggest expenditures tend to buy more items per session (further analysis required). (TODO)  
The other significant correlations don't seem particularly interesting.

In [ ]:
threshold = 0.2
diff_corr = (cust_df_w_out.corr() - cust_df_wo_out.corr()).abs()

for col in diff_corr.columns:
    diff_corr[col] = diff_corr[col].transform(lambda x: x if x>threshold else 0)

print("Difference between the correlation matrices (with and without outliers)")
diff_corr

As noted in the Outlier analysis section, by using the dataset without outliers we lose almost all of the information concerning entries with negative Qta.

## Customer-related outliers analysis

## Remove customer related outliers

In [ ]:
Q1 = cust_df.quantile(0.25)
Q3 = cust_df.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
#fig_height = cust_df.columns.size
#fig = plt.figure(figsize=(20, 30)) 
#fig_dims = (fig_height, 2)
#fig.subplots_adjust(hspace=0.2, wspace=0.2)
#index = 0

for feature in cust_df.columns:
    #plt.subplot2grid(fig_dims, (index, 0))
    #cust_df.boxplot(feature)

    cust_df = cust_df.loc[(cust_df[feature] >= Q1[feature] - 1.5*IQR[feature]) & (cust_df[feature] <= Q3[feature] + 1.5*IQR[feature])]
    
    #plt.subplot2grid(fig_dims, (index, 1))
    #cust_df.boxplot(feature)
    #index = index + 1
    
#plt.show()

Final result

In [ ]:
pd.plotting.scatter_matrix(cust_df, figsize=(10,10))
plt.show()

In [ ]:
cust_df.corr()

The correlation between Iu and Eweekday is even stronger after removing the outliers.  
In order to reduce the dimensionality of the data we will drop Eweekday given the strong similarity to Iu.

In [ ]:
cust_df = cust_df.drop("Eweekday", axis=1)

## Task end
Save the customer features for the next task.

In [ ]:
cust_df_w_out.to_csv("customer_features.csv", sep='\t', index=False)